In [27]:
def add_city(db):
    # Changes to this function will be reflected in the output. 
    # All other functions are for local use only.
    # Try changing the name of the city to be inserted
    #db.cities.insert({"name" : "Chicago"})
    db.cities.insert({"name" : "Moscow",
                     'year':1888})
    
def get_city(db):
    return db.cities.find_one_and_delete({'name':'Moscow'})

def get_db():
    # For local use
    from pymongo import MongoClient
    client = MongoClient('mongodb://localhost:27017/')
    # 'examples' here is the database name. It will be created if it does not exist.
    db = client.examples
    return db

if __name__ == "__main__":
    # For local use
    db = get_db() # uncomment this line if you want to run this locally
    add_city(db)
    print get_city(db)

{u'_id': ObjectId('55898fe2165dfc495573e8d4'), u'name': u'Moscow', u'year': 1888}


In [67]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [8]:
client

MongoClient('localhost', 2701)

In [68]:
db = client.examples

In [16]:
def add_city(db):
    # Changes to this function will be reflected in the output. 
    # All other functions are for local use only.
    # Try changing the name of the city to be inserted
    db.cities.insert({"name" : "Chicago"})
    db.cities.insert({"name" : "Moscow",
                     'year':1888})

In [1]:
from datetime import datetime
from sys import stdout
def extractComponents(createdAt, tzOffset):
    date = datetime.utcfromtimestamp(createdAt+(tzOffset-60)*60)
    return date

In [2]:
def getMongoDBInstance():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.habidatum
    return db
import pandas as pd
import numpy as np
from pymongo import MongoClient
db = getMongoDBInstance()

In [3]:
def getActivitiesFromDB(db):
    activityProjection = {"_id":0,
                          "activity.activities.items.content.object.venue.id":1,
                          "activity.activities.items.content.object.venue.name":1,
                          "activity.activities.items.content.object.id":1,
                          "activity.activities.items.content.object.venue.stats":1,
                          "activity.activities.items.content.object.venue.categories":1,
                            "activity.activities.items.content.object.venue.location.lat":1,
                            "activity.activities.items.content.object.venue.location.lng":1,
                            "activity.activities.items.content.object.checkin.venue.id":1,
                            "activity.activities.items.content.object.checkin.venue.name":1,
                             "activity.activities.items.content.object.checkin.venue.location.lat":1,
                             "activity.activities.items.content.object.checkin.venue.location.lng":1,
                             "activity.activities.items.createdAt":1,
                              "requestedUserId":1}
    activityQuery = {"activity.activities.items.content.type":"checkin"}
    usersVenues = db.activities.find(activityQuery, activityProjection)
    return usersVenues

def saveDataToCSV(checkinsForUsers):
    plainListOfCheckins = [item for sublist in checkinsForUsers for item in sublist]
    plainListOfCheckinsDF = pd.DataFrame(plainListOfCheckins)
    plainListOfCheckinsDF.to_csv('data/July/extendedCheckinsHistory.tsv', sep='\t',encoding='utf-8')
    
#Transfer the data to python objects
def getActivitiesObjects(usersVenues):
    tried = 0
    checkinsForUsers = []
    for userActivity in enumerate(usersVenues):
        checkinsForUser = []
        for checkIn in userActivity[1]['activity']['activities']['items']:
            checkinInfo = {}
            checkinInfo['userId'] = userActivity[1]['requestedUserId']
            checkinInfo["createdAt"] = checkIn['createdAt']
            checkinInfo["createdAtDate"] = str(extractComponents(checkIn['createdAt'], 240))#Get the real time zone

            if "venue" in checkIn['content']['object'].keys():
                checkinInfo["venueId"] = checkIn['content']['object']["venue"]["id"]
                if tried == 0:
                    print checkIn['content']['object']["id"]
                    tried = 1
                checkinInfo['checkinId'] = checkIn['content']['object']["id"]
                checkinInfo["name"] = checkIn['content']['object']["venue"]["name"]
                checkinInfo["lat"] = checkIn['content']['object']["venue"]["location"]["lat"]
                checkinInfo["lng"] = checkIn['content']['object']["venue"]["location"]["lng"]
                
                checkinInfo["checkinsCount"] = checkIn['content']['object']["venue"]["stats"]["checkinsCount"]
                checkinInfo["tipCount"] = checkIn['content']['object']["venue"]["stats"]["tipCount"]
                checkinInfo["usersCount"] = checkIn['content']['object']["venue"]["stats"]["usersCount"]
                
                for category in checkIn['content']['object']["venue"]['categories']:
                    if category['primary']:
                        checkinInfo['categoryId'] = category['id']
                        checkinInfo['categoryName'] = category['name']
                
                checkinsForUser.append(checkinInfo)
            elif "venue" in checkIn['content']['object'].keys():
                checkinInfo["venueId"] = checkIn['content']['object']['checkin']["venue"]["id"]
                checkinInfo["name"] = checkIn['content']['object']['checkin']["venue"]["name"]
                checkinInfo["lat"] = checkIn['content']['object']['checkin']["venue"]["location"]["lat"]
                checkinInfo["lng"] = checkIn['content']['object']['checkin']["venue"]["location"]["lng"]
                checkinsForUser.append(checkinInfo)
        checkinsForUsers.append(checkinsForUser)
    return checkinsForUsers

In [4]:
usersVenues = getActivitiesFromDB(db)
checkinsForUsers = getActivitiesObjects(usersVenues)
saveDataToCSV(checkinsForUsers)

5592136d498ea3f73761c68b


In [22]:
venueSample = {u'stats': {u'checkinsCount': 2028, u'tipCount': 18, u'usersCount': 721}, u'name': u'\u0426\u0435\u043d\u0442\u0440\u0430\u043b\u044c\u043d\u0430\u044f \u0433\u043e\u0440\u043e\u0434\u0441\u043a\u0430\u044f \u0431\u043e\u043b\u044c\u043d\u0438\u0446\u0430 \u2116 1', u'contact': {u'phone': u'84955116363', u'formattedPhone': u'8 (495) 511-63-63'}, u'location': {u'city': u'\u041a\u043e\u0440\u043e\u043b\u0451\u0432', u'lat': 55.91927755960679, u'contextGeoId': 72057594038482169L, u'cc': u'RU', u'country': u'\u0420\u043e\u0441\u0441\u0438\u044f', u'contextLine': u'\u041a\u043e\u0440\u043e\u043b\u0451\u0432', u'state': u'\u041c\u043e\u0441\u043a\u043e\u0432\u0441\u043a\u0430\u044f \u043e\u0431\u043b.', u'formattedAddress': [u'\u0443\u043b. \u0426\u0438\u043e\u043b\u043a\u043e\u0432\u0441\u043a\u043e\u0433\u043e, \u0434. 24', u'141070, \u041a\u043e\u0440\u043e\u043b\u0451\u0432'], u'address': u'\u0443\u043b. \u0426\u0438\u043e\u043b\u043a\u043e\u0432\u0441\u043a\u043e\u0433\u043e, \u0434. 24', u'postalCode': u'141070', u'lng': 37.82716155052185}, u'verified': False, u'id': u'4e6cb9a7ae603c15efbda057', u'categories': [{u'pluralName': u'Hospitals', u'primary': True, u'name': u'Hospital', u'shortName': u'Hospital', u'id': u'4bf58dd8d48988d196941735', u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/building/medical_', u'suffix': u'.png'}}], u'like': False}

In [27]:
print venueSample.keys()
print venueSample

[u'like', u'contact', u'stats', u'name', u'verified', u'id', u'categories', u'location']
{u'like': False, u'contact': {u'phone': u'84955116363', u'formattedPhone': u'8 (495) 511-63-63'}, u'stats': {u'tipCount': 18, u'checkinsCount': 2028, u'usersCount': 721}, u'name': u'\u0426\u0435\u043d\u0442\u0440\u0430\u043b\u044c\u043d\u0430\u044f \u0433\u043e\u0440\u043e\u0434\u0441\u043a\u0430\u044f \u0431\u043e\u043b\u044c\u043d\u0438\u0446\u0430 \u2116 1', u'verified': False, u'id': u'4e6cb9a7ae603c15efbda057', u'categories': [{u'pluralName': u'Hospitals', u'primary': True, u'name': u'Hospital', u'shortName': u'Hospital', u'id': u'4bf58dd8d48988d196941735', u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/building/medical_', u'suffix': u'.png'}}], u'location': {u'contextGeoId': 72057594038482169L, u'cc': u'RU', u'postalCode': u'141070', u'formattedAddress': [u'\u0443\u043b. \u0426\u0438\u043e\u043b\u043a\u043e\u0432\u0441\u043a\u043e\u0433\u043e, \u0434. 24', u'141070, \u041a\u043